In [ ]:
!pip install fastai --upgrade

     |████████████████████████████████| 204kB 30.4MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import os
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
from skimage.color import rgb2lab, lab2rgb

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.models.resnet import resnet18
from torch.utils.data import Dataset, DataLoader

from fastai.vision.learner import create_body
from fastai.vision.models.unet import DynamicUnet
from fastai.data.external import untar_data, URLs

In [ ]:
SIZE = 256

In [ ]:
class TrainingDataset(Dataset):
  def __init__(self, paths):
    self.transforms = transforms.Compose([
            transforms.Resize((SIZE, SIZE)),
            transforms.RandomHorizontalFlip(),
        ])
    self.paths = paths

  def __getitem__(self, idx):
    img = Image.open(self.paths[idx]).convert("RGB")
    img = self.transforms(img)
    img = np.array(img)

    lab_img = rgb2lab(img).astype("float32")
    lab_img = transforms.ToTensor()(lab_img)

    L = lab_img[[0], ...] / 50. - 1.
    ab = lab_img[[1, 2], ...] / 110.

    return {"L": L, "ab": ab}

  def __len__(self):
    return len(self.paths)

In [ ]:
class ValidationDataset(Dataset):
  def __init__(self, paths):
    self.transforms = transforms.Compose([
            transforms.Resize((SIZE, SIZE)),
        ])
    self.paths = paths

  def __getitem__(self, idx):
    img = Image.open(self.paths[idx]).convert("RGB")
    img = self.transforms(img)
    img = np.array(img)

    lab_img = rgb2lab(img).astype("float32")
    lab_img = transforms.ToTensor()(lab_img)

    L = lab_img[[0], ...] / 50. - 1.
    ab = lab_img[[1, 2], ...] / 110.

    return {"L": L, "ab": ab}

  def __len__(self):
    return len(self.paths)

In [ ]:
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride):
    super().__init__()

    self.conv = nn.Sequential(
      nn.Conv2d(
          in_channels, out_channels, 4, stride, 1, bias=False, padding_mode="reflect"
      ),
      nn.BatchNorm2d(out_channels),
      nn.LeakyReLU(0.2),
    )

  def forward(self, x):
    return self.conv(x)

In [ ]:
class PatchDiscriminator(nn.Module):
  def __init__(self, x_channels, y_channels, features=[64, 128, 256, 512]):
    super().__init__()

    self.initial = nn.Sequential(
      nn.Conv2d(x_channels + y_channels, features[0], 4, 2, 1, padding_mode="reflect"),
      nn.LeakyReLU(0.2),
    )

    layers = []
    for i in range(1, len(features)):
      layers.append(
        CNNBlock(features[i-1], features[i], 1 if features[i] == features[-1] else 2),
      )

    layers.append(
      nn.Conv2d(features[-1], 1, 4, 1, 1, padding_mode="reflect"),
    )

    self.model = nn.Sequential(*layers)

  def forward(self, xy):
    xy = self.initial(xy)

    return self.model(xy)

In [ ]:
class GANLoss(nn.Module):
  def __init__(self, real_label=1., fake_label=0.):
    super().__init__()

    self.register_buffer("real_label", torch.tensor(real_label))
    self.register_buffer("fake_label", torch.tensor(fake_label))

    self.loss = nn.BCEWithLogitsLoss()

  def get_labels(self, preds, target_is_real):
    if target_is_real:
      return self.real_label.expand_as(preds)

    return self.fake_label.expand_as(preds)

  def __call__(self, preds, target_is_real):
    labels = self.get_labels(preds, target_is_real)

    return self.loss(preds, labels)

In [ ]:
class MainModel(nn.Module):
  def __init__(self, device, D, G, lr_G=2e-4, lr_D=2e-4, beta1=.5, beta2=.999, lambda_L1=100.):
    super().__init__()

    self.device = device

    self.D = D
    self.G = G

    self.GANcriterion = GANLoss().to(self.device)
    self.L1criterion = nn.L1Loss()

    self.opt_G = optim.Adam(self.G.parameters(), lr=lr_G, betas=(beta1, beta2))
    self.opt_D = optim.Adam(self.D.parameters(), lr=lr_D, betas=(beta1, beta2))

    self.lambda_L1 = lambda_L1

  def set_requires_grad(self, model, requires_grad):
    for parameter in model.parameters():
      parameter.requires_grad = requires_grad

  def setup_input(self, data):
    self.L = data["L"].to(self.device)
    self.ab = data["ab"].to(self.device)

  def forward(self):
    self.fake_color = self.G(self.L).to(device)

  def backward_D(self):
    fake_image = torch.cat([self.L, self.fake_color], dim=1)
    fake_preds = self.D(fake_image.detach())

    self.loss_D_fake = self.GANcriterion(fake_preds, False)

    real_image = torch.cat([self.L, self.ab], dim=1)
    real_preds = self.D(real_image)

    self.loss_D_real = self.GANcriterion(real_preds, True)

    self.loss_D = (self.loss_D_fake + self.loss_D_real) * .5

    self.loss_D.backward()

  def backward_G(self):
    fake_image = torch.cat([self.L, self.fake_color], dim=1)
    fake_preds = self.D(fake_image)

    self.loss_G_GAN = self.GANcriterion(fake_preds, True)
    self.loss_G_L1 = self.lambda_L1 * self.L1criterion(self.fake_color, self.ab)

    self.loss_G = self.loss_G_GAN + self.loss_G_L1

    self.loss_G.backward()

  def optimize(self):
    self.forward()

    self.D.train()
    self.set_requires_grad(self.D, True)
    self.opt_D.zero_grad()
    self.backward_D()
    self.opt_D.step()

    self.G.train()
    self.set_requires_grad(self.D, False)
    self.opt_G.zero_grad()
    self.backward_G()
    self.opt_G.step()

In [ ]:
class AverageMeter:
  def __init__(self):
    self.reset()

  def reset(self):
    self.count, self.avg, self.sum = 0., 0., 0.

  def update(self, val, count):
    self.count += count
    self.sum += val * count
    self.avg = self.sum / self.count

In [ ]:
def init_loss_meters():
  loss_D_fake = AverageMeter()
  loss_D_real = AverageMeter()
  loss_D = AverageMeter()

  loss_G_GAN = AverageMeter()
  loss_G_L1 = AverageMeter()
  loss_G = AverageMeter()

  return {"loss_D_fake": loss_D_fake,
          "loss_D_real": loss_D_real,
          "loss_D": loss_D,
          "loss_G_GAN": loss_G_GAN,
          "loss_G_L1": loss_G_L1,
          "loss_G": loss_G}

def update_losses(model, loss_meters, count):
  for loss_name, loss_meter in loss_meters.items():
    loss = getattr(model, loss_name)
    loss_meter.update(loss.item(), count)

def log_results(loss_meters):
  for loss_name, loss_meter in loss_meters.items():
    print(f"{loss_name}: {loss_meter.avg:.4f}")

In [ ]:
def lab_to_rgb(L, ab):
  L = (L + 1.) * 50.
  ab = ab * 110.

  Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()

  rgb_imgs = []
  for img in Lab:
    img_rgb = lab2rgb(img)
    rgb_imgs.append(img_rgb)

  return np.stack(rgb_imgs, axis=0)

def visualize(model, data, lead, save=True):
  model.G.eval()

  with torch.no_grad():
    model.setup_input(data)
    model.forward()

  model.G.train()

  fake_color = model.fake_color.detach()
  real_color = model.ab
  L = model.L

  fake_imgs = lab_to_rgb(L, fake_color)
  real_imgs = lab_to_rgb(L, real_color)

  fig = plt.figure(figsize=(15, 8))
  for i in range(5):
    ax = plt.subplot(3, 5, i + 1)
    ax.imshow(L[i][0].cpu(), cmap="gray")
    ax.axis("off")
    ax = plt.subplot(3, 5, i + 1 + 5)
    ax.imshow(fake_imgs[i])
    ax.axis("off")
    ax = plt.subplot(3, 5, i + 1 + 10)
    ax.imshow(real_imgs[i])
    ax.axis("off")
  plt.show()

  if save:
    fig.savefig(f"./images/colorization_{lead}_{time.time()}.png")

In [ ]:
class Block(nn.Module):
  def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
    super(Block, self).__init__()

    self.conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
      if down
      else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
    )

    self.use_dropout = use_dropout
    self.dropout = nn.Dropout(0.5)
    self.down = down

  def forward(self, x):
    x = self.conv(x)

    return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
  def __init__(self, in_channels=1, out_channels=2, features=64):
    super().__init__()

    self.initial_down = nn.Sequential(
      nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
      nn.LeakyReLU(0.2),
    )
    self.down1 = Block(features, features * 2, True, "leaky", False)
    self.down2 = Block(features * 2, features * 4, True, "leaky", False)
    self.down3 = Block(features * 4, features * 8, True, "leaky", False)
    self.down4 = Block(features * 8, features * 8, True, "leaky", False)
    self.down5 = Block(features * 8, features * 8, True, "leaky", False)
    self.down6 = Block(features * 8, features * 8, True, "leaky", False)
    self.bottleneck = nn.Sequential(nn.Conv2d(features * 8, features * 8, 4, 2, 1), nn.ReLU())

    self.up1 = Block(features * 8, features * 8, False, act="relu", use_dropout=True)
    self.up2 = Block(features * 8 * 2, features * 8, False, "relu", True)
    self.up3 = Block(features * 8 * 2, features * 8, False, "relu", True)
    self.up4 = Block(features * 8 * 2, features * 8, False, "relu", False)
    self.up5 = Block(features * 8 * 2, features * 4, False, "relu", False)
    self.up6 = Block(features * 4 * 2, features * 2, False, "relu", False)
    self.up7 = Block(features * 2 * 2, features, False, "relu", False)
    self.final_up = nn.Sequential(
      nn.ConvTranspose2d(features * 2, out_channels, 4, 2, 1),
      nn.Tanh(),
    )

  def forward(self, x):
    d1 = self.initial_down(x)
    d2 = self.down1(d1)
    d3 = self.down2(d2)
    d4 = self.down3(d3)
    d5 = self.down4(d4)
    d6 = self.down5(d5)
    d7 = self.down6(d6)
    bottleneck = self.bottleneck(d7)

    up1 = self.up1(bottleneck)
    up2 = self.up2(torch.cat([up1, d7], 1))
    up3 = self.up3(torch.cat([up2, d6], 1))
    up4 = self.up4(torch.cat([up3, d5], 1))
    up5 = self.up5(torch.cat([up4, d4], 1))
    up6 = self.up6(torch.cat([up5, d3], 1))
    up7 = self.up7(torch.cat([up6, d2], 1))

    return self.final_up(torch.cat([up7, d1], 1))

In [ ]:
root = str(untar_data(URLs.COCO_SAMPLE)) + "/train_sample"

paths = glob.glob(root + "/*.jpg")

np.random.seed(42)
paths_subset = np.random.choice(paths, 12_000, replace=False)

rand_idxs = np.random.permutation(12_000)
train_idxs = rand_idxs[:10_000]
val_idxs = rand_idxs[10_000:]

train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]

train_dset = TrainingDataset(train_paths)
val_dset = ValidationDataset(val_paths)

train_dl = DataLoader(train_dset, 16, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_dset, 16, num_workers=2, pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
D = PatchDiscriminator(1, 2).to(device)
G = Generator().to(device)

In [ ]:
model = MainModel(device, D, G)

In [ ]:
def train_model(model, train_dl, val_dl, epochs, display_every=200):
  fixed_val_data = next(iter(val_dl))

  for e in range(epochs):
    loss_meters = init_loss_meters()
    i = 0
    for data in tqdm(train_dl):
      model.setup_input(data)

      model.optimize()

      update_losses(model, loss_meters, data["L"].size(0))

      i += 1
      if i % display_every == 0:
        print(f"\nEpoch {e+1}/{epochs}")
        print(f"Iteration {i}/{len(train_dl)}")
        log_results(loss_meters)

        visualize(model, fixed_val_data, e)

    torch.save(model.D.state_dict(), f"./models/{e}_D_{time.time()}.pt")
    torch.save(model.G.state_dict(), f"./models/{e}_G_{time.time()}.pt")

In [ ]:
train_model(model, train_dl, val_dl, 100)

KeyboardInterrupt: ignored